In [1]:
import glob
import pickle
import re
import os
from IPython.display import clear_output

In [2]:
baseDir = 'C:\\Users\\applied informatics\\pipVirtualEnvironments\\nlpVenv\\workContents\\protocolDocumentsText\\'
trainData = []                    #Training Data
entityType = 'INVESTIGATOR'

In [3]:
protoFileNameList = glob.glob(baseDir + 'Batch2\\*.txt')
print("[INFO]: Number of protocol documents: ", len(protoFileNameList))

[INFO]: Number of protocol documents:  37


In [4]:
DATARANGE = 300

In [5]:
'''
Format of annotated document in spacy training data for NER
("DOCUMENT_TEXT", {"entities": [(START_INDEX, START_INDEX + LENGTH_OF_ENTITY, "ENTITY_TYPE"),
                               (START_INDEX, START_INDEX + LENGTH_OF_ENTITY, "ENTITY_TYPE"),
                               (START_INDEX, START_INDEX + LENGTH_OF_ENTITY, "ENTITY_TYPE"),
                               ...]})                               
'''

'\nFormat of annotated document in spacy training data for NER\n("DOCUMENT_TEXT", {"entities": [(START_INDEX, START_INDEX + LENGTH_OF_ENTITY, "ENTITY_TYPE"),\n                               (START_INDEX, START_INDEX + LENGTH_OF_ENTITY, "ENTITY_TYPE"),\n                               (START_INDEX, START_INDEX + LENGTH_OF_ENTITY, "ENTITY_TYPE"),\n                               ...]})                               \n'

In [6]:
piRe = re.compile(r'Investigator')              
pageBreakRe = re.compile(r'>---PAGEBREAK--->')

In [7]:
def searchForBeginOfDoc(data,spanObjectListPBPart):
    '''
    Find index from where to start the document
    Input: 
        1) Entire protocol Document
        2) List, in the reverse order, of spans of page breaks upto the location where a match 
            is found
    Output: Index from where to start the document
    '''
    
    clear_output()
    print("="*100)
    print("[INFO]: Searching for starting index of doc")
    print("="*100)
    input("Press enter to continue: ")
    
    # if there is no page break before this match then this is the first page, 
    #then this doc starts from the start of entire protocol doc
    if len(spanObjectListPBPart) == 0:
        print("="* 100)
        print("Begin index of doc: ", 0)
        print("="*100)
        input("Press Enter to continue: ")
        return 0
    
    for spanObject in spanObjectListPBPart:
        clear_output()
        
        # Candidate text for starting the document begins with upper case letter
        #if data[spanObject[1] + 1].isupper():
        
        if DATARANGE < spanObject[1]:
            print("="*100)
            print(data[spanObject[1] + 1-DATARANGE:spanObject[1]+DATARANGE])
            print("="*100)
            tentativeStartIndex = spanObject[1] + 1-DATARANGE
            
        else:
            print("="*100)
            print(data[:spanObject[1]+DATARANGE])
            print("="*100)
            tentativeStartIndex = 0
            
        answer = input("Ok to start document from some position within this content?(y/n): ")

        while answer not in ['y', 'n']:
            print("You did not enter a valid response!")
            answer = input("Enter a valid response(y/n): ")

        if answer == 'y':
            startText = input("Enter some text from where the doc should start")
            beginIndex = tentativeStartIndex + data[tentativeStartIndex:spanObject[1] + DATARANGE].find(startText)
            clear_output()
            print("="*100)
            print("Found beginning index of document: ", beginIndex)
            print("="*100)
            input("Press enter to continue: ")
            return beginIndex
    
    # if no suitable starting index is found, then this doc starts from the start of entire 
    #protocol doc
    print("="* 100)
    print("Begin index of doc: ", 0)
    print("="*100)
    input("Press Enter to continue: ")
    return 0

In [8]:
def searchForEndOfDoc(data,spanObjectListPbFullDoc,beginLoc,start):
    '''
    Find index at which to end the document
    Input: 
        1) Entire protocol Document
        2) List of span objects for occurrences of >---PAGEBREAK---> in the entire protocol
         doc
        3) Index at which match occurred
        4) Starting point for search of end of document
    Output: Index at which to end the document
    '''
    
    clear_output()
    print("="*100)
    print("[INFO]: Beginning search for end index of doc")
    print("="*100)
    input("Press enter to continue")
    
    # If no further page break occurs, this doc ends at where the entire protocol doc ends
    if start == (-1,-1):
        clear_output()
        print('='*100)
        print('End of doc: ', len(data))
        print('='*100)
        input("Press enter to continue: ")
        return len(data)
    
    while True:
        clear_output()
        print("="*100)
        print(data[beginLoc:start[0]])
        print("="*100)
        answer = input("Are all entities covered in the portion of data displayed above?(y/n): ")
        while answer not in ['y', 'n']:
            print("You did not enter a valid response!")
            answer = input("Enter a valid response(y/n): ")

        if answer == 'y':
            clear_output()
            print("[INFO]: Will include upto(if they exist) next two pages in the document")
            
            if (spanObjectListPbFullDoc.index(start) + 2) < len(spanObjectListPbFullDoc):
                print('='*100)
                print('[INFO]: End index of doc: ', spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start) + 2][1])
                print('='*100)
                input("Press enter to continue")
                return spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start) + 2][1]
            
            elif (spanObjectListPbFullDoc.index(start) + 1) < len(spanObjectListPbFullDoc):
                print('='*100)
                print('[INFO]: End index of doc: ', spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start) + 1][1])
                print('='*100)
                input("Press enter to continue")
                return spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start) + 1][1]
            
            else:
                print('='*100)
                print('[INFO]: End index of doc: ', spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start)][1])
                print('='*100)
                input("Press enter to continue")
                return spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start)][1]

        # if all entities are not covered, try to extend the portion of doc    
        else:
            if (spanObjectListPbFullDoc.index(start) + 1) < len(spanObjectListPbFullDoc):
                start = spanObjectListPbFullDoc[spanObjectListPbFullDoc.index(start) + 1]
            
    # if no suitable end index is found, this doc ends at where the entire protocol doc ends
    print('='*100)
    print('[INFO]:End index of doc: ', len(data))
    print('='*100)
    input("Press enter to continue: ")
    return len(data)

In [9]:
def findDocSpan(data,spanObjectListPbFullDoc,beginLoc,spanObjectListPBPart,immediateNextPageBreakSpan):
    '''
    Find span for current doc
    Input:
    
    Output:
    Span of this doc
    '''
    
    startIndexOfDoc = searchForBeginOfDoc(data,spanObjectListPBPart)
    endIndexOfDoc = searchForEndOfDoc(data,spanObjectListPbFullDoc,beginLoc,immediateNextPageBreakSpan)
    print("Document span for this doc: ",(startIndexOfDoc,endIndexOfDoc))
    input("Press enter to continue")
    
    return (startIndexOfDoc,endIndexOfDoc)

In [10]:
def extractDoc(data,docSpanIndex):
    '''
    Extract a portion from the entire protocol doc as defined by input span
    Input:
        1)
        2)
    Output:
        Protocol document portion as defined by the input span
    '''
    
    beginDoc = int(docSpanIndex[0:docSpanIndex.find('-')])
    endDoc = int(docSpanIndex[docSpanIndex.find('-')+1:])
    return data[beginDoc:endDoc]

In [11]:
def getSpanObjectListPb(data):
    matchObjectIteratorPb = pageBreakRe.finditer(data)
    spanObjectListPb = [matchObjectPb.span() for matchObjectPb in matchObjectIteratorPb]
    return spanObjectListPb

In [12]:
def createAndSaveDocs(data,finalDocSpanIndexSet,trainData,protoFileName):
    clear_output()
    print("[INFO]: Final document span set: ", finalDocSpanIndexSet)
    input("Press enter to continue")
    count = 0
    for docSpanIndex in finalDocSpanIndexSet:
        print("[INFO]: Beginning extraction of data in the span: ",docSpanIndex)
        
        doc = extractDoc(data,docSpanIndex)
        count += 1
        print("[INFO]: Created document from data in the span: ", docSpanIndex)
        fileName = os.path.basename(protoFileName) + '(' + str(count) + ')'
        with open('C:\\Users\\applied informatics\\pipVirtualEnvironments\\nlpVenv\\workContents\\extractedProtocolDocuments\\' + fileName,'w') as f:
            f.write(doc)

In [13]:
#Find what portions of each protocol document to include
def findDocPortions():
    filesProcessed = 0
    for protoFileName in protoFileNameList:
        clear_output()
        print("="*100)
        print("[INFO]: Processing file: ", protoFileName)
        print("="*100)
        input("Press enter to contine")
        
        with open(protoFileName, 'r') as f:
            data = f.read()
        
        # Get list of span objects for occurrences of >---PAGEBREAK---> in the entire protocol
        # doc
        spanObjectListPbFullDoc = getSpanObjectListPb(data)

        candidateDocSpanIndexSet = set()
        
        #Word Investigator may occur at multiple places in the document
        #Get all match objects where the word Investigator occurs
        matchObjectIteratorPI = piRe.finditer(data)
        spanObjectListPI = [matchObject.span() for matchObject in matchObjectIteratorPI]
        
        currentMatchObjectIndex = -1
        nextMatchObjectIndex = currentMatchObjectIndex + 1
        
        matchCount = 0
        docSpan = (-1,-1)
        #For each place where the word Investigator occurs
        for matchObject in spanObjectListPI:
            if docSpan != (-1,-1):
                clear_output()
                print("[INFO]: Recent doc span added: ", docSpan)
                input("Press enter to continue")
            matchCount += 1
            currentMatchObjectIndex += 1
            nextMatchObjectIndex = currentMatchObjectIndex + 1
            #If there is another match object
            if nextMatchObjectIndex < len(spanObjectListPI):
                # if the span object for the next match lies within the range doc span added recently
                # then it must be that all the entities of the current match lie within data in the doc span
                # added recently. So, the entities of current match will not result in new doc
                print("Next match Object: ",spanObjectListPI[nextMatchObjectIndex] )
                input("Press enter to continue")
                if spanObjectListPI[nextMatchObjectIndex][0] > docSpan[0] and spanObjectListPI[nextMatchObjectIndex][1] < docSpan[1]:
                    print("[INFO]: Match #",matchCount, "  for the word Investigator at index: ",
                         matchObject[0], " will not yield a new doc. So continuing to next match")
                    input("Press enter to continue")
                    continue
            clear_output()
            print("[INFO]: Match #", matchCount, " for the word Investigator at index: ",matchObject[0])
            print("[INFO]: Here is some portion of data around the matching index")
            
            if DATARANGE <= matchObject[0]:
                print("="*100)
                print(data[matchObject[0]-DATARANGE:matchObject[0]+DATARANGE])
                print("="*100)
                beginLoc = matchObject[0]-DATARANGE
                
            else:
                print("="*100)
                print(data[:matchObject[0]+DATARANGE])
                print("="*100)
                beginLoc = 0
            
            seeMoreData = 0
            while(True):
                resp = input("Need to see more data?(y/n):  ")
                while resp not in ['y', 'n']:
                    print("You entered an invalid response!")
                    resp = input("Enter a valid response(y/n): ")
                    
                # if the printed data is not enough to enter entities, print more    
                if resp == 'y':
                    clear_output()
                    seeMoreData += 100
                    print(data[beginLoc:beginLoc+DATARANGE+seeMoreData])
                else:
                    break
            
            # Check if the match has entities and use it only if it has entities
            hasEntity = input("Does the match have entity(ies) in it?(y/n): ")
            while hasEntity not in ['y','n']:
                print("You entered an invalid response!")
                hasEntity = input("Enter a valid response(y/n): ")
                
            if hasEntity == 'y':
                #beginLoc = matchObject.span()[0]
                immediateNextPageBreak = pageBreakRe.search(data[matchObject[0]:]) # Starting point for search of end of document

                if immediateNextPageBreak is not None:
                    immediateNextPageBreakSpan = (matchObject[0] + immediateNextPageBreak.span()[0],
                                             matchObject[0] + immediateNextPageBreak.span()[1])

                else:
                    immediateNextPageBreakSpan = (-1,-1)

                #Get page break spans from the beginning of protocol doc upto the match
                spanObjectListPBPart = getSpanObjectListPb(data[:beginLoc])
                spanObjectListPBPart.reverse()
                
                docSpan = findDocSpan(data,spanObjectListPbFullDoc,beginLoc,spanObjectListPBPart,immediateNextPageBreakSpan)

                candidateDocSpanIndexSet.add(docSpan)

        clear_output()
        print('='*100)
        print("[INFO]: Here is the candidate span set for the file: ")
        print(candidateDocSpanIndexSet)
        print("="*100)
        input("Press enter to continue")

        if candidateDocSpanIndexSet != set():
            finalDocSpanIndexSet = set(input("Enter final span(s) separated by commas with each span of the format START-END: ").split(','))
            #Check if span is in valid format
            #checkFinalDocSpanIndexSet(finalDocSpanIndexSet)
            createAndSaveDocs(data,finalDocSpanIndexSet,trainData,protoFileName)
            filesProcessed += 1
            print("[INFO]: Number of files processed: ", filesProcessed)
            input("Press enter to continue")
            
        else:
            clear_output()
            filesProcessed += 1
            print('='*100)
            print("[INFO]: Number of files processed: ", filesProcessed)
            print('[INFO]: No entity found in this protocol document')
            print('='*100)
            input("Press enter to continue")

In [ ]:
findDocPortions()

[INFO]: Final document span set:  {'164743-167141', '11793-20517', '0-4243', '36067-44918'}
Press enter to continue
[INFO]: Beginning extraction of data in the span:  164743-167141
[INFO]: Created document from data in the span:  164743-167141
[INFO]: Beginning extraction of data in the span:  11793-20517
[INFO]: Created document from data in the span:  11793-20517
[INFO]: Beginning extraction of data in the span:  0-4243
[INFO]: Created document from data in the span:  0-4243
[INFO]: Beginning extraction of data in the span:  36067-44918
[INFO]: Created document from data in the span:  36067-44918
[INFO]: Number of files processed:  27
